In [1]:
from lib import *

data = read_datacsv("./data.csv", nrows=100000)
data.head()

,open,high,low,close,volume
datetime,,,,,
2017-01-02 17:03:00,2364.50,2365.00,2364.50,2364.75,209
2017-01-02 17:04:00,2364.75,2364.75,2364.50,2364.75,114
2017-01-02 17:05:00,2364.50,2365.75,2364.50,2365.75,479
2017-01-02 17:06:00,2365.75,2365.75,2365.25,2365.25,207
2017-01-02 17:07:00,2365.50,2365.50,2364.50,2364.75,164


In [3]:
vv = generate_volumebars(data, 10000)
vv

,high,low,close,volume
datetime,,,,
2017-01-02 17:03:00,2366.25,2361.75,2362.50,9971
2017-01-02 19:42:00,2369.00,2362.50,2368.25,9932
2017-01-03 00:48:00,2371.25,2368.25,2371.00,10062
2017-01-03 02:13:00,2373.00,2370.25,2371.00,10012
2017-01-03 02:55:00,2372.75,2369.50,2371.75,9870
...,...,...,...,...
2017-04-17 15:15:00,2470.25,2468.25,2469.00,11620
2017-04-17 17:41:00,2471.50,2468.00,2468.25,9807
2017-04-17 20:52:00,2469.50,2467.75,2469.50,10218


In [4]:
labeled_data = labelling(vv, length=50, dynamic=True, period=30, beta=1.5)
labeled_data

,high,low,close,volume,labels
datetime,,,,,
2017-01-02 17:03:00,2366.25,2361.75,2362.50,9971,0
2017-01-02 19:42:00,2369.00,2362.50,2368.25,9932,0
2017-01-03 00:48:00,2371.25,2368.25,2371.00,10062,0
2017-01-03 02:13:00,2373.00,2370.25,2371.00,10012,0
2017-01-03 02:55:00,2372.75,2369.50,2371.75,9870,0
...,...,...,...,...,...
2017-04-13 15:07:00,2450.75,2448.50,2450.50,11085,1
2017-04-13 15:13:00,2451.25,2450.00,2450.75,4389,1
2017-04-13 15:15:00,2452.50,2447.25,2450.50,15535,1


In [5]:
label_counts = labeled_data['labels'].value_counts()
print(label_counts)


labels
 1    2564
-1    2258
 0      57
Name: count, dtype: int64


In [8]:
class labeler:

    def __init__(self, data:pd.DataFrame, vol_thresholds:list, horizons:list, betas:list, periods:list):
        self.data = data
        self.vol_thresholds = vol_thresholds
        self.horizons = horizons
        self.betas = betas
        self.periods = periods

    def labelling_ATR(self, df, horizon, beta, period):
        atr = calculate_atr(self.data, period)
        uper_barrier = df.high + beta*atr
        lower_barrier = df.low - beta*atr

        labels = list()
        labeled_data = df.copy()

        
        for i in range(len(df)-horizon):

            series = df[i:i+horizon]
            pre_labels =  np.where(series.high >= uper_barrier.iloc[i], 1,
                           (np.where(series.low <= lower_barrier.iloc[i], -1, 0)))
        
        # Si hay alguna etiqueta distinta de 0,
        # quiero la primera en suceder.
            if any(pre_labels != 0):
                labels.append(pre_labels[pre_labels != 0][0])
            else:
                labels.append(0)
        labeled_data.drop(labeled_data.index[-horizon:],axis=0, inplace=True) # Pierdo los ultimos "length" datos
        labeled_data["labels"] = np.array(labels)    

        return labeled_data
    
    def labelling_volat(self, df, horizon, beta, period):
        volat = df.close.pct_change()
        volat = volat.rolling(period).std()
        uper_barrier = df.high * (1+beta*volat)
        lower_barrier = df.low - (1+beta*volat)
        
        labels = list()
        labeled_data = df.copy()

        for i in range(len(df)-horizon):
            
            series = df[i:i+horizon]
            pre_labels =  np.where(series.high >= uper_barrier.iloc[i], 1,
                           (np.where(series.low <= lower_barrier.iloc[i], -1, 0)))
        # Si hay alguna etiqueta distinta de 0,
        # quiero la primera en suceder.
            if any(pre_labels != 0):
                labels.append(pre_labels[pre_labels != 0][0])
            else:
                labels.append(0)
        labeled_data.drop(labeled_data.index[-horizon:],axis=0, inplace=True) # Pierdo los ultimos "length" datos
        labeled_data["labels"] = np.array(labels)    

        return labeled_data
    

    def run_labeler(self):
        output = dict()
        for i in self.vol_thresholds:
            df = generate_volumebars(self.data, i)

            ### ACA HAY QUE LLAMAR FUNCION QUE AGREGA LOS FEATURES

            for h in self.horizons:
                for b in self.betas:
                    for p in self.periods:
                        labels_atr = self.labelling_ATR(df, h, b, p)
                        output[f"v_{i}, h_{h}, b_{b}, p_{p}, ATR"] = labels_atr
                        labels_volat = self.labelling_volat(df, h, b, p)
                        output[f"v_{i}, h_{h}, b_{b}, p_{p}, volat"] = labels_volat
        
        return output

In [9]:
lab = labeler(data, [5000, 10000], [50, 100], [1, 1.5], [14, 50])


In [10]:
dicc = lab.run_labeler()

In [26]:
for i in dicc.values():
    print(len(i))

9411
9411
9411
9411
9411
9411
9411
9411
9361
9361
9361
9361
9361
9361
9361
9361
4879
4879
4879
4879
4879
4879
4879
4879
4829
4829
4829
4829
4829
4829
4829
4829
